# Trilateration: Analysis

Given a set of RTT samples annotated by probe and anchor ID, and metadata about anchors including location, determine the linear fit between 

### Needful things

First, imports, utility functions, etc. required for analysis, and thaw out the datastore

In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import math

from collections import namedtuple

In [ ]:
with pd.HDFStore('rtt.h5') as store:
    anchor_df = store['anchor_df']
    probe_df = store['probe_df']
    rtt_df = store['rtt_df']

Calculate minumum RTT per anchor/probe pair, then join anchor and probe location to this dataframe

In [ ]:
min_df = pd.DataFrame({'minrtt': rtt_df.groupby(['aid','pid'])['rtt'].min()})
loc_by_aid = anchor_df.loc[:,('lon','lat')]
loc_by_aid.columns = ['alon','alat']
loc_by_pid = probe_df.loc[:,('lon','lat')]
loc_by_pid.columns = ['plon','plat']
min_df = loc_by_pid.join(loc_by_aid.join(min_df, how="inner"), how="inner")

Now calculate distance with a vectorization of the unit sphere distance algorithm.

In [ ]:
min_df['tmp_aphi'] = (90.0 - min_df['alat']) * math.pi/180.0
min_df['tmp_pphi'] = (90.0 - min_df['plat']) * math.pi/180.0
min_df['tmp_atheta'] = min_df['alon'] * math.pi/180.0
min_df['tmp_ptheta'] = min_df['plon'] * math.pi/180.0
min_df['tmp_cos'] = ( np.sin(min_df['tmp_aphi']) * np.sin(min_df['tmp_pphi']) * 
                      np.cos(min_df['tmp_atheta'] - min_df['tmp_ptheta']) + 
                      np.cos(min_df['tmp_aphi']) * np.cos(min_df['tmp_pphi']) )
min_df['km'] = np.arccos(min_df['tmp_cos']) * 6371
del min_df['tmp_aphi']
del min_df['tmp_pphi']
del min_df['tmp_atheta']
del min_df['tmp_ptheta']
del min_df['tmp_cos']

### Trilateration

Given three points and distances, estimate a location.

In [ ]:
def on_unit_sphere(long1, lat1, long2, lat2):
 
    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0
         
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
         
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
         
    # Compute spherical distance from spherical coordinates.
         
    # For two locations in spherical coordinates 
    # (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) = 
    #    sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length
     
    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) + 
           math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )
 
    return arc

def earth_surface_km(long1, lat1, long2, lat2):
    # Remember to multiply arc by the radius of the earth 
    # in your favorite set of units to get length.
    return on_unit_sphere(lat1, long1, lat2, long2) * 6371

def trilaterate(lonA, latA, distA, lonB, latB, distB, lonC, latC, distC):
    """
    Given three latitudes, longitudes, and distances, 
    estimate a point by trilateration. 
    
    Assumes a spherical Earth of radius 6371 km, and all elevations 0.
    
    Shamelessly stolen from https://gis.stackexchange.com/questions/66/trilateration-using-3-latitude-and-longitude-points-and-3-distances
    """

    #assuming elevation = 0
    earthR = 6371

    #using authalic sphere
    #if using an ellipsoid this step is slightly different
    #Convert geodetic Lat/Long to ECEF xyz
    #   1. Convert Lat/Long to radians
    #   2. Convert Lat/Long(radians) to ECEF
    xA = earthR *(math.cos(math.radians(latA)) * math.cos(math.radians(lonA)))
    yA = earthR *(math.cos(math.radians(latA)) * math.sin(math.radians(lonA)))
    zA = earthR *(math.sin(math.radians(latA)))

    xB = earthR *(math.cos(math.radians(latB)) * math.cos(math.radians(lonB)))
    yB = earthR *(math.cos(math.radians(latB)) * math.sin(math.radians(lonB)))
    zB = earthR *(math.sin(math.radians(latB)))

    xC = earthR *(math.cos(math.radians(latC)) * math.cos(math.radians(lonC)))
    yC = earthR *(math.cos(math.radians(latC)) * math.sin(math.radians(lonC)))
    zC = earthR *(math.sin(math.radians(latC)))

    P1 = np.array([xA, yA, zA])
    P2 = np.array([xB, yB, zB])
    P3 = np.array([xC, yC, zC])

    #from wikipedia
    #transform to get circle 1 at origin
    #transform to get circle 2 on x axis
    ex = (P2 - P1)/(np.linalg.norm(P2 - P1))
    i = np.dot(ex, P3 - P1)
    ey = (P3 - P1 - i*ex)/(np.linalg.norm(P3 - P1 - i*ex))
    ez = np.cross(ex, ey)
    d = np.linalg.norm(P2 - P1)
    j = np.dot(ey, P3 - P1)

    #from wikipedia
    #plug and chug using above values
    x = (pow(distA,2) - pow(distB,2) + pow(d,2))/(2*d)
    y = ((pow(distA,2) - pow(distC,2) + pow(i,2) + pow(j,2))/(2*j)) - ((i/j)*x)

    # only one case shown here
    z = np.sqrt(pow(distA,2) - pow(x,2) - pow(y,2))

    #triPt is an array with ECEF x,y,z of trilateration point
    triPt = P1 + x*ex + y*ey + z*ez

    lat = math.degrees(math.asin(triPt[2] / earthR))
    lon = math.degrees(math.atan2(triPt[1],triPt[0]))

    return (lon, lat)

def dist_km(minrtt_us):
    """
    A function to estimate the distance in kilometers given a minimum RTT in microseconds.
    
    For now, do this very simply: multiply the speed of light in 
    kilometers per microsecond by a factor and RTT/2
    """
    
    c = 0.299792458
    linfit_factor = 0.37 # derived from linear fit of Atlas RTT data
    
    return (minrtt_us * c * linfit_factor) / 2

In [ ]:
fast_min_df = min_df[min_df['minrtt'] < 40000]

for i in range(100):
    A = fast_min_df.iloc[i]
    B = fast_min_df.iloc[i+1]
    C = fast_min_df.iloc[i+2]
    
    if A.name[0] == B.name[0] == C.name [0]:
        try:
            (xalon, xalat) = trilaterate(A.plon, A.plat, dist_km(A.minrtt), 
                                         B.plon, B.plat, dist_km(B.minrtt), 
                                         C.plon, C.plat, dist_km(C.minrtt))
            print("error: {}".format(earth_surface_km(xalon, xalat, A.alon, A.alat)))
        except ValueError:
            pass
        